In [68]:
import numpy as np
import math


def f(x):  # Define the objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

A = np.array(([5, 3], [3, 5]), dtype=float)
b = np.array([8*math.sqrt(2.), 8*math.sqrt(2.)])

eigs = np.linalg.eigvals(A)
print("The eigenvalues of A:", eigs)

if (np.all(eigs>0)):
    print("A is positive definite")
elif (np.all(eigs>=0)):
    print("A is positive semi-definite")
else:
    print("A is negative definite")

if (A.T==A).all()==True: print("A is symmetric")

def linear_CG(x, A, b, epsilon):
    res = A.dot(x) - b  # Initialize the residual
    delta = -res  # Initialize the descent direction

    while True:

        if np.linalg.norm(res) <= epsilon:
            return x, f(x)  # Return the minimizer x* and the function value f(x*)

        D = A.dot(delta)
        beta = -(res.dot(delta)) / (delta.dot(D))  # Line (11) in the algorithm
        x = x + beta * delta  # Generate the new iterate

        res = A.dot(x) - b  # generate the new residual
        chi = res.dot(D) / (delta.dot(D))  # Line (14) in the algorithm
        delta = chi * delta - res  # Generate the new descent direction


linear_CG(np.array([0, 0]), A, b, 10**-5)

The eigenvalues of A: [8. 2.]
A is positive definite
A is symmetric


(array([1.41421356, 1.41421356]), 0.0)

In [69]:
from autograd import grad
import autograd.numpy as np
import math
from scipy.optimize import line_search
NORM = np.linalg.norm

def func(x): # Objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

Df = grad(func) # Gradient of the objective function

def Fletcher_Reeves(Xj, tol, alpha_1, alpha_2):
    x1 = [Xj[0]]
    x2 = [Xj[1]]
    D = Df(Xj)
    delta = -D # Initialize the descent direction
    
    while True:
        start_point = Xj # Start point for step length selection 
        beta = line_search(f=func, myfprime=Df, xk=start_point, pk=delta, c1=alpha_1, c2=alpha_2)[0] # Selecting the step length
        if beta!=None:
            X = Xj+ beta*delta #Newly updated experimental point
        
        if NORM(Df(X)) < tol:
            x1 += [X[0], ]
            x2 += [X[1], ]
            return X, func(X) # Return the results
        else:
            Xj = X
            d = D # Gradient at the preceding experimental point
            D = Df(Xj) # Gradient at the current experimental point
            chi = NORM(D)**2/NORM(d)**2 # Line (16) of the Fletcher-Reeves algorithm
            delta = -D + chi*delta # Newly updated descent direction
            x1 += [Xj[0], ]
            x2 += [Xj[1], ]
            
print(Fletcher_Reeves(np.array([-1.7, -3.2]), 10**-5, 10**-4, 0.38))

(array([0.70710678, 0.70710678]), -8.000000000000002)


In [70]:
from autograd import grad
import autograd.numpy as np
import math
from scipy.optimize import line_search
NORM = np.linalg.norm

def func(x): # Objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

Df = grad(func) # Gradient of the objective function

def Polak_Ribiere(Xj, tol, alpha_1, alpha_2):
    x1 = [Xj[0]]
    x2 = [Xj[1]]
    D = Df(Xj)
    delta = -D # Initialize the descent direction
    
    while True:
        start_point = Xj # Start point for step length selection 
        beta = line_search(f=func, myfprime=Df, xk=start_point, pk=delta, c1=alpha_1, c2=alpha_2)[0] # Selecting the step length
        if beta!=None:
            X = Xj+ beta*delta # Newly updated experimental point 
        
        if NORM(Df(X)) < tol:
            x1 += [X[0], ]
            x2 += [X[1], ]
            return X, func(X) # Return the results
        else:
            Xj = X
            d = D # Gradient of the preceding experimental point
            D = Df(Xj) # Gradient of the current experimental point
            chi = (D-d).dot(D)/NORM(d)**2 
            chi = max(0, chi) # Line (16) of the Polak-Ribiere Algorithm
            delta = -D + chi*delta # Newly updated direction
            x1 += [Xj[0], ]
            x2 += [Xj[1], ]
            
print(Polak_Ribiere(np.array([-1.7, -3.2]), 10**-6, 10**-4, 0.2))

(array([0.70710678, 0.70710678]), -8.0)


In [71]:
from autograd import grad
import autograd.numpy as np
import math
from scipy.optimize import line_search
NORM = np.linalg.norm

def func(x): # Objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

Df = grad(func) # Gradient of the objective function

def Hestenes_Stiefel(Xj, tol, alpha_1, alpha_2):
    x1 = [Xj[0]]
    x2 = [Xj[1]]
    D = Df(Xj)
    delta = -D
    
    while True:
        start_point = Xj # Start point for step length selection 
        beta = line_search(f=func, myfprime=Df, xk=start_point, pk=delta, c1=alpha_1, c2=alpha_2)[0] # Selecting the step length
        if beta!=None:
            X = Xj+ beta*delta
        
        if NORM(Df(X)) < tol:
            x1 += [X[0], ]
            x2 += [X[1], ]
            return X, func(X)
        else:
            Xj = X
            d = D
            D = Df(Xj)
            chi = D.dot(D - d)/delta.dot(D - d) # See line (16)
            delta = -D + chi*delta
            x1 += [Xj[0], ]
            x2 += [Xj[1], ]

print(Hestenes_Stiefel(np.array([-1.7, -3.2]), 10**-6, 10**-4, 0.2))

(array([0.70710678, 0.70710678]), -8.0)


In [72]:
from autograd import grad
import autograd.numpy as np
import math
from scipy.optimize import line_search
NORM = np.linalg.norm

def func(x): # Objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

Df = grad(func) # Gradient of the objective function

def Dai_Yuan(Xj, tol, alpha_1, alpha_2):
    x1 = [Xj[0]]
    x2 = [Xj[1]]
    D = Df(Xj)
    delta = -D
    
    while True:
        start_point = Xj # Start point for step length selection 
        beta = line_search(f=func, myfprime=Df, xk=start_point, pk=delta, c1=alpha_1, c2=alpha_2)[0] # Selecting the step length
        if beta!=None:
            X = Xj+ beta*delta
        
        if NORM(Df(X)) < tol:
            x1 += [X[0], ]
            x2 += [X[1], ]
            return X, func(X)
        else:
            Xj = X
            d = D
            D = Df(Xj)
            chi = NORM(D)**2/delta.dot(D - d) # See line (16)
            delta = -D + chi*delta
            x1 += [Xj[0], ]
            x2 += [Xj[1], ]
            
print(Dai_Yuan(np.array([-1.7, -3.2]), 10**-6, 10**-4, 0.2))

(array([0.70710678, 0.70710678]), -8.000000000000002)


In [73]:
from autograd import grad
import autograd.numpy as np
import math
from scipy.optimize import line_search
NORM = np.linalg.norm

def func(x): # Objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

Df = grad(func) # Gradient of the objective function

def Hager_Zhang(Xj, tol, alpha_1, alpha_2):
    x1 = [Xj[0]]
    x2 = [Xj[1]]
    D = Df(Xj)
    delta = -D
    
    while True:
        start_point = Xj # Start point for step length selection 
        beta = line_search(f=func, myfprime=Df, xk=start_point, pk=delta, c1=alpha_1, c2=alpha_2)[0] # Selecting the step length
        if beta!=None:
            X = Xj+ beta*delta
        
        if NORM(Df(X)) < tol:
            x1 += [X[0], ]
            x2 += [X[1], ]
            return X, func(X)
        else:
            Xj = X
            d = D
            D = Df(Xj)
            Q = D - d
            M = Q - 2*delta*NORM(Q)**2/(delta.dot(Q))
            N = D/(delta.dot(Q))
            chi = M.dot(N) # See line (19)
            delta = -D + chi*delta
            x1 += [Xj[0], ]
            x2 += [Xj[1], ]
            
print(Hager_Zhang(np.array([-1.7, -3.2]), 10**-6, 10**-4, 0.2))

(array([0.70710678, 0.70710678]), -8.000000000000002)


In [74]:
from autograd import grad
import autograd.numpy as np
from scipy.optimize import minimize
import math

def func(x): # Objective function
    return 5*x[0]**2 + 6*x[0]*x[1] + 5*x[1]**2 - 8*math.sqrt(2)*x[0] - 8*math.sqrt(2)*x[1]

Df = grad(func) # Gradient of the objective function

res=minimize(fun=func, x0=np.array([-9., 4.]), jac=Df, method='CG', options={'gtol':10**-6, 'disp':True, 'return_all':True})

cnt = 1 # counter
for i in res.allvecs: 
    print(i)
    cnt = cnt + 1
    # if cnt == 100:
        # break;

Optimization terminated successfully.
         Current function value: -8.000000
         Iterations: 3
         Function evaluations: 7
         Gradient evaluations: 7
[-9.  4.]
[-4.20069713  5.57136627]
[0.7676896  0.77699143]
[0.70710678 0.70710678]
